In [1]:
from Functions import *
from Optimizations import *
from utils import *
from Saving import *

%load_ext autoreload

%autoreload 2

In [2]:
import time
import yaml

In [6]:
# g_params = [[-10, 1, -2], [0, 10, -1], [10, 3, 1], [-20, 1, 3], [20, 1, 3]]
# g_params = [[-10, 0.7, -1], [0, 10, -1.5], [9, 3, 1], [17.5, 10, -0.5], [-20, 1, 3], [25, 1, 3]]
g_params = [[-40, 0.7, -2],[-30, 0.7, -1.5],[-5, 0.7, -1], [35, 0.7, -1.5], 
            [-17.5, 30, -3], [5, 10, -1.5], [14, 3, 1], [25, 10, -0.5], 
            [-50, 1, 3], [50, 1, 3]]

process = {}

process["particle_init"] = {"name": "1d_uniform", 
                            "num_particles": 5000}

process["potential_function"] = {"name": "gaussian", 
                                 "params": g_params}

process["total_iter"] = 50
process["tau"] = 50
process["x_range"] = [-25, 25]

process["gamma"] = 0.1
process["temperature"] = 0.5
process["epsilon"] = 0

process["weight_function"] = {"name": "norm"} 
process["resample_function"] = {"name": "softmax", "params": {"beta": 0.5}} 

process["domain_enforcer"] = {"name": "hyper_cube_enforcer", "params": {"strength": 0.2}}

all_paths = diffusion_resampling(process, return_full_path=False)


0.44879309428608855
0.7212484265563016
0.7654524577124526
0.779833210512062
0.7773651848784473
0.7779496456674386
0.773609596583744
0.7772423271419578
0.7831276388238335
0.7756602980108181
0.7819549569821472
0.7796747478646563
0.7722925320849242
0.7684400181068154
0.7744838404590537
0.768594446880883
0.7809677633957164
0.7752118724118242
0.7687961988728431
0.774883175933849
0.7714340157293047
0.7784910475241565
0.7702309585006931
0.7813233352266679
0.7760506385805169
0.7772945019553179
0.7784560355023604
0.7701132802417954
0.7776209553859811
0.777100072364413
0.7636816442255596
0.7702334713321057
0.7794315144198805
0.7688135092831797
0.7768646650903015
0.776297587687458
0.7827497533100939
0.7788004760828546
0.7793860592704637
0.7677781943024901
0.7730934541002552
0.7715036278164791
0.7812049574771972
0.7743200510816428
0.7742360424837951
0.7764437929265093
0.7661992712476583
0.7772611840338647
0.7746834891041008
0.7786129020499274


In [7]:
# want to get order: big_timestep, tau_timestep, particle, point

f, grad_f = get_potential(process)

X = np.linspace(process["x_range"][0], process["x_range"][1], 200)
inp = np.array([X])
Y = f(inp)

inp = all_paths.reshape([np.product(all_paths.shape[:3]), all_paths.shape[-1]]).T
out = f(inp)
all_paths_proc = np.concatenate([inp.T, out.reshape([len(out), 1])], axis=1).reshape(all_paths.shape[0], all_paths.shape[1], all_paths.shape[2], 2)

In [ ]:
# full process densities 
K = multi_gaussian(np.array([[0.6]]))

folder_name = "full_{}_{}".format(process["potential_function"]["name"], process["tau"])
saved_path = create_animation_1d_pictures_particles(all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)import plotly.graph_objects as go
save_config(saved_path, process)

In [8]:
# only feynman kac after selection densities 
fk_all_paths_proc = all_paths_proc[:, :, 0, :]
fk_all_paths_proc = fk_all_paths_proc.reshape([fk_all_paths_proc.shape[0], fk_all_paths_proc.shape[1], 1, fk_all_paths_proc.shape[2]])

K = multi_gaussian(np.array([[0.6]]))

folder_name = "selection_{}_{}_{}".format(process["potential_function"]["name"], process["resample_function"]["params"]["beta"], process["tau"])
saved_path = create_animation_1d_pictures_particles(fk_all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)
save_config(saved_path, process)

In [121]:
# only feynman kac after mutation densities 
fk_all_paths_proc = all_paths_proc[:, :, -1, :]
fk_all_paths_proc = fk_all_paths_proc.reshape([fk_all_paths_proc.shape[0], fk_all_paths_proc.shape[1], 1, fk_all_paths_proc.shape[2]])

K = multi_gaussian(np.array([[0.6]]))

folder_name = "mutation_{}_{}_{}".format(process["potential_function"]["name"], process["resample_function"]["params"]["beta"], process["tau"])
saved_path = create_animation_1d_pictures_particles(fk_all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)
save_config(saved_path, process)

### Things to note:

The beta value of the softmax turns out to be very important. Adjusting that value determines how much to value each respective shallow/flat regions. Seems like changing it can give you a nice stationary distribution around certain falt minima. 

Also the tau value is important. Letting the run run for too long will cause the particles to approach more or less the stationary distribution if ran with soley diffusion. 

Maybe penalize where you started from. 



In [1]:
def load_config(load_dir):
    with open(load_dir + "/process.yml", "r") as f:
        c = yaml.load(f)
    return c

c = load_config("./tmp/1d__1579540892.8095877")

NameError: name 'yaml' is not defined